# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [128]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. 
'''

## 2. Generate the mathematical model

In [145]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [146]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

### Objective Function (Maximize Daily Profit)

The objective is to maximize the daily profit, which is the difference between income and cost. 

**Profit = Income - Cost**

**Income:** 49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 

**Cost:** 150 * H (where H is the number of hectoliters of syrup used)

We need to express H in terms of X1 and X2. From the problem statement, we know:

* 1 hectoliter of syrup produces 40 units of COLI (X1) and 20 units of PEPSA (X2)
* Therefore, H = (X1/40) + (X2/20)

Substituting this into the cost function:

**Cost:** 150 * ((X1/40) + (X2/20))

Therefore, the objective function to maximize is:

**Maximize:** 49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * ((X1/40) + (X2/20))

### Constraints

1. **Production Capacity:** The total number of 1.5L products produced per day cannot exceed the production line capacity. The capacity is 7100 units per hour, and the firm operates 8 hours per day, 5 days a week. However, since we are optimizing daily profit, we will consider only the daily capacity.

**(X1 + X2) <= 7100 * 8**

2. **Minimum PEPSA Production:** The firm is committed to producing at least half the amount of PEPSA compared to COLI.

**X2 >= 0.5 * X1**

3. **Non-negativity:** The production quantities cannot be negative.

**X1 >= 0**
**X2 >= 0**

### Model Summary

**Maximize:** 49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * ((X1/40) + (X2/20))

**Subject to:**

* X1 + X2 <= 56800 
* X2 >= 0.5 * X1
* X1 >= 0
* X2 >= 0

This model can be solved using various optimization software or tools to find the optimal values of X1 and X2 that maximize the daily profit while satisfying the production constraints. 


## 3. Generate the pyomo code

In [147]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [148]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a Pyomo ConcreteModel
model = pyo.ConcreteModel()

# Define decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of COLI
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of PEPSA

# Define objective function (Maximize profit)
model.profit = pyo.Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * (
        (model.X1 / 40) + (model.X2 / 20)
    ),
    sense=pyo.maximize
)

# Define constraints
model.capacity_constraint = pyo.Constraint(
    expr=model.X1 + model.X2 <= 56800  # Daily production capacity
)
model.production_ratio_constraint = pyo.Constraint(
    expr=model.X2 >= 0.5 * model.X1  # Minimum PEPSA production
)

# Solve the model using a solver (e.g., CBC)
solver = pyo.SolverFactory('cbc')
solver.solve(model)

# Print the optimal solution
print("Optimal Production Quantities:")
print("COLI (X1):", pyo.value(model.X1))
print("PEPSA (X2):", pyo.value(model.X2))
print("Maximum Daily Profit:", pyo.value(model.profit))
```

**Explanation:**

1. **Model Creation:** We create a `ConcreteModel` instance to represent our optimization problem. 
2. **Decision Variables:** We define two decision variables, `X1` and `X2`, representing the production quantities of COLI and PEPSA respectively. They are defined as non-negative real numbers.
3. **Objective Function:** The objective function `profit` is defined to maximize the daily profit based on the provided formula. 
4. **Constraints:**
    * `capacity_constraint`: This constraint ensures that the total daily production does not exceed the capacity of 56,800 units (7100 units/hour * 8 hours).
    * `production_ratio_constraint`: This constraint ensures that the production of PEPSA is at least half the production of COLI.
5. **Solver:** We choose the `cbc` solver (you can use other solvers like glpk, cplex, gurobi) to solve the model. 
6. **Solution and Output:** The optimal values for `X1`, `X2`, and the maximum profit are printed.

**Sample Data:**

* The production capacity (56,800 units) is calculated based on the given information about production line capacity (7100 units/hour) and daily operating hours (8 hours).
* The coefficients in the objective function and constraints are derived from the problem statement. 


## 4. Input problem data and try running the generated code

In [152]:
import pyomo.environ as pyo

# Create a Pyomo ConcreteModel
model = pyo.ConcreteModel()

# Define decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of COLI
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of PEPSA

# Define objective function (Maximize profit)
model.profit = pyo.Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * (
        (model.X1 / 40) + (model.X2 / 20)
    ),
    sense=pyo.maximize
)

# Define constraints
model.capacity_constraint = pyo.Constraint(
    expr=model.X1 + model.X2 <= 56800  # Daily production capacity
)
model.production_ratio_constraint = pyo.Constraint(
    expr=model.X2 >= 0.5 * model.X1  # Minimum PEPSA production
)

# Solve the model using a solver (e.g., CBC)
solver = pyo.SolverFactory('ipopt')
solver.solve(model)

# Print the optimal solution
print("Optimal Production Quantities:")
print("COLI (X1):", pyo.value(model.X1))
print("PEPSA (X2):", pyo.value(model.X2))
print("Maximum Daily Profit:", pyo.value(model.profit))

Optimal Production Quantities:
COLI (X1): 16335.833333340004
PEPSA (X2): 8167.91666666004
Maximum Daily Profit: 400289176.04199195


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [153]:
print(response.text)

## Mathematical Optimization Model for Production Planning

### Objective Function (Maximize Daily Profit)

The objective is to maximize the daily profit, which is the difference between income and cost. 

**Profit = Income - Cost**

**Income:** 49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 

**Cost:** 150 * H (where H is the number of hectoliters of syrup used)

We need to express H in terms of X1 and X2. From the problem statement, we know:

* 1 hectoliter of syrup produces 40 units of COLI (X1) and 20 units of PEPSA (X2)
* Therefore, H = (X1/40) + (X2/20)

Substituting this into the cost function:

**Cost:** 150 * ((X1/40) + (X2/20))

Therefore, the objective function to maximize is:

**Maximize:** 49000 * X1 - X1^2 + 30 * X2 - 2 * X2^2 - 150 * ((X1/40) + (X2/20))

### Constraints

1. **Production Capacity:** The total number of 1.5L products produced per day cannot exceed the production line capacity. The capacity is 7100 units per hour, and the firm operates 8 hours per day, 5 days a wee

In [154]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a Pyomo ConcreteModel
model = pyo.ConcreteModel()

# Define decision variables
model.X1 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of COLI
model.X2 = pyo.Var(domain=pyo.NonNegativeReals)  # Production quantity of PEPSA

# Define objective function (Maximize profit)
model.profit = pyo.Objective(
    expr=49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2 - 150 * (
        (model.X1 / 40) + (model.X2 / 20)
    ),
    sense=pyo.maximize
)

# Define constraints
model.capacity_constraint = pyo.Constraint(
    expr=model.X1 + model.X2 <= 56800  # Daily production capacity
)
model.production_ratio_constraint = pyo.Constraint(
    expr=model.X2 >= 0.5 * model.X1  # Minimum PEPSA production
)

# Solve the model using a solver (e.g., CBC)
solver = pyo.SolverFactory('cbc')
solver.solve(model)

# Print the optimal solution
print("Optimal Production Quantities:")
print("COLI (X1):", pyo.value(model.X1))
print("PEPSA (X